# Minitarea 2

Nombre: Joaquín Ignacio Pérez Araya

---------------------------
## Language Models (3 pts)
Estas preguntas son teóricas y deben ser resueltas con desarrollo, pero sin código. Por favor usen $\LaTeX$ para las fórmulas. El material del curso y los videos deberian ser suficientes para que puedan responder.


### Pregunta 1 (1 pt)
Asuma que tiene un modelo de lenguaje de trigramas (simple) entrenado sobre un corpus C. Muestre cómo el modelo le asigna una probabilidad a la oración `el perro le ladra al gato` usando los parámetros estimados a partir del corpus  $q(w_i | w_{i-2}, w_{i-1})$ .

**Respuesta:**

Se adiciona el token especial `STOP` a los tokens de la oración.
Así lo que se calcula es: $P(el,perro,le,ladra,al,gato,STOP)$.

Se le agregan los tokens extras de Padding, así:

$P(*,*,el,perro,le,ladra,al,gato,STOP)$

Se utiliza regla de la cadena y usando un modelo de Markov de Segundo Orden:

$P(el|*,*) \times P(perro|*,el) \times P(le|el,perro) \times P(ladra|perro,le) \times $
$P(al| le,ladra) \times P(gato|ladra,al) \times P(STOP| al,gato )$

Estas son equivalentes a los parámetros obtenidos del corpus $C$:
$q(el|*,*) \times q(perro|*,el) \times q(le|el,perro) \times q(ladra|perro,le) \times $
$q(al| le,ladra) \times q(gato|ladra,al) \times q(STOP| al,gato )$

### Pregunta 2 (1 pt)
Muestre cómo se calcularía  $q(w_{i} | w_{i-2}, w_{i-1})$  usando un modelo que interpola un modelo de lenguajes de trigramas, bigramas, y unigrama.

**Respuesta:**

Supongamos que se tienen 3 modelos de lenguajes: Uno con Trigramas ($T$), otro con Bigramas ($B$) y uno con Unigramas ($U$), 
con sus respectivas probabilidades $q_T(w_i|w_{i-2},w_{i-1})$, $q_B(w_i|w_{i-1})$ y $q_U(w_i|w_{i-1})$ respectivamente, que pueden ser calculados de diferentes formas.

Sea $\lambda_1,\lambda_2,\lambda_3\ge 0$ tal que $\lambda_1+\lambda_2+\lambda_3 = 1$

$q(w_{i} | w_{i-2}, w_{i-1})$ será calculado a partir de un interpolación lineal de las probabilidades de mencionadas anteriomente, es decir: 

$q(w_{i} | w_{i-2}, w_{i-1}) = \lambda_1 \cdot q_U(w_{i}) + \lambda_2 \cdot q_B(w_{i}|w_{i-1}) + \lambda_3 \cdot q_B(w_{i}|w_{i-2},w_{i-1}) $

### Pregunta 3 (1 pt)
¿Qué ventajas tiene el modelo interpolado sobre el modelo de trigramas simple?

**Respuesta:**
El modelo interpolado es más flexible a los casos cuando se le calculan probablidades de oraciones (en particular trigramas) que no existen en el Corpus de entrenamiento. 
De esta forma no se da probabilidad $0$ a oraciones que tengan 
secuencias de palabras que no se hayan visto nunca en el entrenamiento, que es factible que ocurra en realidad.



-----------------------
## Naive Bayes (3 pts)
En esta parte de la minitarea deberan programar Naive Bayes Multinomial usando Laplace Smothing. Las referencias las pueden encontrar en el material del curso y los videos del profesor.

A continuacion se presentan un conjunto de documentos de training divididos en 2 categorias distintas. Ustedes deberan clasificar los documentos del test set usando Naive Bayes con Laplace Smothing.

Por favor, documenten su codigo. Escriban que hacen las funciones, que reciben, que entregan, etc. Si en algun lugar de su codigo hacen algo que creen que debe ser explicado, pongan comentarios, son bienvenidos. 


**Underflow prevention:** En vez de hacer muchas multiplicacions de `float`s, reemplacenlas por sumas de logaritmos para prevenir errores de precision. Revisen la diapo 69 de las slides. 

NOTA: Sobre las `namedtuple`s. Es el tipo de los documentos. Son objetos inmutable que tienen dos atributos: `words` donde estan las palabras del documento y `class_` donde se guarda la clase de ese documento. Estos objetos son inmutables, lo que quiere decir que si quieren modificar un documento y cambiarle la clase, tienen que crear otro documento. Otra cosa es que son tuplas como cualquier otra, es decir se pueden acceder usando indices como `doc[0]` o `doc[1]`. Son libres de implementar su solucion como quieran, si no les gusta este tipo de dato usen otro.



In [8]:
from pprint import pprint
from collections import namedtuple


document = namedtuple(
    "document", ("words", "class_")  # avoid python's keyword collision
)

train_set = (
    document(words=('w03', 'w01', 'w02', 'w06', 'w02', 'w08', 'w07'), class_=0),
    document(words=('w05', 'w04', 'w00', 'w06', 'w09', 'w07', 'w06', 'w09', 'w05'), class_=1),
    document(words=('w07', 'w06', 'w00', 'w08', 'w01', 'w08', 'w08', 'w09', 'w02'), class_=0),
    document(words=('w08', 'w09', 'w02', 'w06', 'w05', 'w08', 'w07'), class_=1),
    document(words=('w09', 'w08', 'w05', 'w08', 'w05', 'w00', 'w08'), class_=1),
    document(words=('w05', 'w05', 'w06', 'w01', 'w06', 'w08', 'w02'), class_=1),
    document(words=('w04', 'w03', 'w07', 'w05', 'w04', 'w00', 'w02'), class_=0),
    document(words=('w01', 'w00', 'w01', 'w04', 'w09', 'w02', 'w04', 'w07'), class_=1)
)

print("Train documents:")
pprint(train_set)



test_set = (document(words=('w02', 'w09', 'w06', 'w01', 'w05', 'w04', 'w03', 'w03'), class_=None),)
print("\nTest documents:")
pprint(test_set)


Train documents:
(document(words=('w03', 'w01', 'w02', 'w06', 'w02', 'w08', 'w07'), class_=0),
 document(words=('w05', 'w04', 'w00', 'w06', 'w09', 'w07', 'w06', 'w09', 'w05'), class_=1),
 document(words=('w07', 'w06', 'w00', 'w08', 'w01', 'w08', 'w08', 'w09', 'w02'), class_=0),
 document(words=('w08', 'w09', 'w02', 'w06', 'w05', 'w08', 'w07'), class_=1),
 document(words=('w09', 'w08', 'w05', 'w08', 'w05', 'w00', 'w08'), class_=1),
 document(words=('w05', 'w05', 'w06', 'w01', 'w06', 'w08', 'w02'), class_=1),
 document(words=('w04', 'w03', 'w07', 'w05', 'w04', 'w00', 'w02'), class_=0),
 document(words=('w01', 'w00', 'w01', 'w04', 'w09', 'w02', 'w04', 'w07'), class_=1))

Test documents:
(document(words=('w02', 'w09', 'w06', 'w01', 'w05', 'w04', 'w03', 'w03'), class_=None),)


La implementación se realizó creando una clase que implementa Naive Bayes.

In [16]:
import pandas as pd
import numpy as np

class Naive_Bayes:
    count_data = pd.DataFrame() # Conteo de palabras bajo una clase
    classes = [] # Rotulos de clases
    vocab = []   # Palabras del Vocabulario
    class_count = [] # Conteo total de clases
    word_count = []  # Conteo total de palabras
    
    def __init__(self, train):
        # Get classes + Get Vocab
        for T in train:
            if not T.class_ in self.classes:
                self.classes += [T.class_]
                
            for w in T.words:
                if not w in self.vocab:
                    self.vocab += [w]
            
        # Create dataframe
        self.count_data = pd.DataFrame(columns=self.vocab, index=self.classes)
        
        # Count total words ocurrences and class ocurrences
        self.word_count = np.zeros(len(self.vocab))
        self.class_count = np.zeros(len(self.classes))         
        for T in train:
            class_index = self.classes.index(T.class_)
            self.class_count[class_index] += 1
            for w in T.words:
                word_index = self.vocab.index(w)
                self.word_count[word_index] += 1
        
        # Count words with their respective class
        for c in self.classes:
            count = np.zeros(len(self.vocab))
            for T in train:
                if T.class_ == c:
                    for w in T.words:
                        count[self.vocab.index(w)] += 1
            self.count_data.loc[c] = count


    def class_prob(self, class_): # P(c)
        class_index = self.classes.index(class_)
        return self.class_count[class_index] / sum(self.class_count)
            

    def word_prob(self, word, c): # P(w|c)
        word_index = self.vocab.index(word)
        word_class_count = self.count_data.loc[c][word_index]
        word_total_count = self.word_count[word_index]
        # Retorno la probablidad con LaPlace Smoothing
        return (word_class_count + 1) / (word_total_count + len(self.vocab))

    def get_class(self, words): 
        classValue = np.zeros(len(self.classes))
        # Por cada clase calcula el P(c|d=w1, w2,...,wn)
        for c in self.classes:
            class_index = self.classes.index(c)
            # Calculo su valor que es la suma de los logs
            classValue[class_index] += np.log2(self.class_prob(c))
            for w in words:
                classValue[class_index] += np.log2(self.word_prob(w,c))
        
        # Retorno la que tiene mayor score
        return self.classes[np.argmax(classValue)]
                        
myBayes = Naive_Bayes(train_set)
myBayes.get_class(test_set[0].words)

-20.49685377738804